In [2]:
import pickle
import re
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer


In [3]:
with open('articles-pickle/articles_0.pickle', 'rb') as f:
    queries = pickle.load(f)

In [4]:
articles_with_lowercase_titles = [
    {**article, 'ArticleTitle': [title.lower() for title in article.get('ArticleTitle', [])]}
    for article in queries
]

In [5]:
def info_exc(article):
    title = article["ArticleTitle"]
    title = ''.join(title).replace('\n', ' ').strip()
    author = article["Author"]
    author = ''.join(author).replace('\n', ' ').strip()
    year = article["Date"]
    year = ''.join(year)[:4]
    references = article["References"]
    return title, author, year, references

In [8]:
title, author,year,references = info_exc(queries[21])

In [9]:
def pdf_to_text(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [14]:
text=pdf_to_text("C:/Users/boran/Desktop/Local-Citation-Recommendation-main/pdfs/test.pdf")

In [15]:
def split_article(article_text):
    reference_pattern = r"Kaynakça\n|Kaynaklar\n|References\n|Bibliography\n|Citations\n|Sonnotlar\n"
    reference_match = re.search(reference_pattern, article_text, re.IGNORECASE)

    if reference_match:
        main_text = article_text[:reference_match.start()] #extract main text from references
        reference_section = article_text[reference_match.start():]

        return main_text.replace('\n', ' ')
    else:
        return article_text.replace('\n', ' ')

text = split_article(text)

In [16]:
pattern_to_remove =  r'\b(\d+)\s+([\w\s]+)\((\d{4})\)\.\s*([^\.]+)\.\s*([^,]+),\s*(\d+)\s*\((\d+)\),\s*(\d+)-(\d+)\.\b'
matches = re.finditer(pattern_to_remove, text)

# Initialize a count variable
count = 0

# Iterate through the matches and count them
for match in matches:
    count += 1
    matched_sentence = match.group()
    print(matched_sentence)
    print("\n--------------\n")

# Print the total count of matching sentences
print(f"Total matches found: {count}")

159 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

160 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

161 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

162 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

163 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

164 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

165 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye 

In [17]:
text = re.sub(pattern_to_remove, '', text)
text

'Türkiye’de Olağanüstü Hal:   Devlet Aklı, Askerler ve Siviller Güven Gürkan Öztan , İstanbul Üniversitesi Siyasal Bilgiler Fakültesi,  e-posta: guven.oztan@istanbul.edu.tr. Egemen B. Bezci, Nottingham Üniversitesi Tarih Bölümü,   e-posta: ahxebbe@nottingham.ac.uk. Özet 12 Eylül 1980 askeri darbesi Türkiye’de sosyal, ekonomik ve siyasal alanda geniş çaplı  değişikliklere yol açmıştır. Aynı zamanda darbe yönetimi, 1982 yılında yürürlü ğe giren  ve günümüzde de geçerliliğini sürdüren Anayasa’yı hazırlamıştır.  1982 anayasanın  önemli özelliklerinden biri olağanüstü hal kavramını Türkiye’nin siyasi arenasına  taşıması olmuştur. 1982 Anayasası ile birlikte anayasal düzene dâhil olan olağanüstü  hal, mevcut sıkıyönetim uygulamalarının yerine geçerek 1 Mart 1984 tarihinden  başlayarak uygulamadan kalktığı 30 Kasım 2002 tarihine kadar varlığını sürdürmüştür.  Bu çalışma, olağanüstü hal uygulamasının devlet aklının bir tezahürü olduğunu iddia  etmektedir. Tehdit algılamalarına göre şekillenen 

In [18]:
def window(text, loc, target_str, left_window_size=150, right_window_size=100):
    left_threshold = max(loc - left_window_size, 0)
    right_threshold = min(loc + right_window_size, len(text))
    return text[left_threshold:right_threshold]


def find_references_with_context(text, title, author, year):
    
    # Find matches within parentheses
    matches_parentheses = re.compile(r'(\(.*?\))')
    # Find matches within square brackets, including the brackets
    matches_square_brackets = re.compile(r'(\[.*?\])')
    
    references_with_context = []

    # Iterate through matches within parentheses
    for match_text in matches_parentheses.finditer(text):
        if match_text.group().lower() != title.lower() and author not in match_text.group() and str(year) not in match_text.group():
            start_loc = text.find(match_text.group())
            context = window(text, start_loc, match_text.group(), 150, 100)
            references_with_context.append(context)

    # Iterate through matches within square brackets
    for match_text in matches_square_brackets.finditer(text):
        if match_text.group().lower() != title.lower() and author not in match_text.group() and str(year) not in match_text.group():
            start_loc = text.find(match_text.group())
            context = window(text, start_loc, match_text.group(), 150, 100)
            references_with_context.append(context)

    return references_with_context

In [19]:
sentences_with_references=find_references_with_context(text, title, author, year)

In [20]:
sentences_with_references = list(set(sentences_with_references))

In [21]:
def extract_titles(references):
    titles = []
    for ref in references:
        # Regex pattern to extract titles within quotation marks
        match = re.search(r'\)\. (.+?)\.', ref)
        if match:
            titles.append(match.group(1).lower())
        else:
            # Fallback patterns for various formats
            patterns = [
                r'\)\. (.*?)\. [A-ZÇĞİÖŞÜ][a-zÇĞİÖŞÜ]+',  # New pattern for title after author/year
                r'(?<=\.\s)([A-ZÇĞİÖŞÜ][^,:()]+?)(?:, çev\.|, ed\.|\s\(\d{4}\))',   # Format 1
                r'\. ([^\.]+)\. The Canadian Journal',                              # Format 2
                r'\. ([^\.]+)\. Journal of [A-Z][a-z]+',                           # Format 3
                r'\. ([^\.]+?)\. [A-ZÇĞİÖŞÜ][a-zÇĞİÖŞÜ]+',                        # Format 4
                r'\. (.+?)\. [A-Z][a-z]+'                                          # General fallback
            ]

            for pattern in patterns:
                match = re.search(pattern, ref)
                if match:
                    titles.append(match.group(1).lower())
                    break

    return titles

In [22]:
def find_matching_articles(titles):
    matched = [query["Abstract"] for query in articles_with_lowercase_titles
               if 'ArticleTitle' in query and
               any(ref in query['ArticleTitle'] for ref in titles)]
    return matched

In [23]:
def integrated_function(sentences_with_references, references):
    def extract_citations(window):
        # Find matches within parantheses, including parantheses
        matches_parentheses = re.findall(r'(\(.*?\))', text)

        # Find matches within square brackets, including the brackets
        matches_square_brackets = re.findall(r'(\[.*?\])', text)
    
        filtered_matches_2 = [match for match in matches_parentheses if (':' in match and ',' in match) or ':' in match or ';' in match or match.isdigit()]
        filtered_matches_3 = [match for match in matches_square_brackets if match[1:-1].isdigit()]
        return filtered_matches_2 + filtered_matches_3

    def calculate_cosine_similarity_tfidf(citation, references):
        all_strings = [citation] + references
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(all_strings)
        similarity_matrix = cosine_similarity(tfidf_matrix)

        best_match_index = similarity_matrix[0, 1:].argmax()
        best_match_reference = references[best_match_index]
        similarity_score = similarity_matrix[0, 1 + best_match_index]

        return best_match_reference, similarity_score

    results = []

    for window in sentences_with_references:
        citations = extract_citations(window)
        for citation in citations:
            matched_reference, similarity_score = calculate_cosine_similarity_tfidf(citation, references)
            if similarity_score > 0.25:
                title = extract_titles([matched_reference])
                matched_article = find_matching_articles(title)
                result = {
                    "window": window,
                    "citation": citation,
                    "reference": matched_reference,
                    "cosine_similarity_score": similarity_score,
                    "title": title,
                    "abstract": matched_article  # Add the matched article to the dictionary
                }
                results.append(result)

    return results

In [24]:
results = integrated_function(sentences_with_references, references)

In [25]:
results

[]